In [7]:
import matplotlib.pyplot as plt
import pandas as pd
import os
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 150)
import seaborn as sns
sns.set_theme(color_codes=True)
from scipy import stats
import statsmodels.api as sm
import numpy as np
import json

/opt/homebrew/lib/python3.9/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
def load_data(folder_address = "./Data/Questionnaire Data/" ):
    list_of_data_names = os.listdir(folder_address)
    data = []
    for i in list_of_data_names:
        if(i != ".DS_Store"):
            df = pd.read_csv(folder_address + i)
            data.append(df)

    return pd.concat(data)
            

    

In [28]:
df = load_data()

In [30]:
len(df.mturkID.unique())

31

In [34]:
questionnaire_raw_data = df
# questionnaire_raw_data = questionnaire_raw_data.dropna(axis=0, how='any')
questionnaire_respones = questionnaire_raw_data[questionnaire_raw_data.trial_type.isin(['table-completion', 'custom-form'])][['mturkID', 'experiment_id', 'trial_type', 'id', 'responses']]


In [35]:
len(questionnaire_respones.mturkID.unique())

31

In [36]:
questionnaire_raw_data

,run_id,condition,rt,url,trial_type,trial_index,time_elapsed,internal_node_id,hitId,workerId,assignmentId,a,tp_a,b,tp_b,c,tp_c,experiment_id,assignmentID,mturkID,hitID,stimulus,response,exp_stage,view_history,exp_part,own_rows,responses,id,single,recorded_at,ip,user_agent,device,browser,browser_version,platform,platform_version,source_code_version
0,34,1,24940.1,https://raw.githubusercontent.com/Sepsad/EDE-q...,external-html,0,25208,0.0-0.0,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,ARWF605I7RWM7,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,YH7I1,YH7I1,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,334281,334281,GNG_0qmcw848,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,ARWF605I7RWM7,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,"""","""","""","""","""","""","""","""","""",2021-12-04 04:32:47,173.23.42.244,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,WebKit,Chrome,96.0.4664.45,Linux,NaN,6356fe4a7118019579648d8725187f92
1,34,1,11531.4,"""",html-button-response,1,37247,0.0-1.0,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,ARWF605I7RWM7,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,YH7I1,YH7I1,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,334281,334281,GNG_0qmcw848,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,ARWF605I7RWM7,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,<img src='../img/welcome.gif' style='height: 2...,0,first_welcome,"""","""","""","""","""","""",2021-12-04 04:32:47,173.23.42.244,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,WebKit,Chrome,96.0.4664.45,Linux,NaN,6356fe4a7118019579648d8725187f92
2,34,1,4040.8,"""",instructions,2,41790,0.0-2.0,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,ARWF605I7RWM7,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,YH7I1,YH7I1,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,334281,334281,GNG_0qmcw848,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,ARWF605I7RWM7,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,"""","""",instructions,"[{""page_index"":0,""viewing_time"":4039.799999998...",instructions,"""","""","""","""",2021-12-04 04:32:47,173.23.42.244,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,WebKit,Chrome,96.0.4664.45,Linux,NaN,6356fe4a7118019579648d8725187f92
3,34,1,30957.0,"""",table-completion,3,73278,0.0-3.0,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,ARWF605I7RWM7,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,YH7I1,YH7I1,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,334281,334281,GNG_0qmcw848,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,ARWF605I7RWM7,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,"""","""","""","""","""",[],"{""responses"":{""1"":{""name"":""1. Have you been de...",first-1,true,2021-12-04 04:32:47,173.23.42.244,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,WebKit,Chrome,96.0.4664.45,Linux,NaN,6356fe4a7118019579648d8725187f92
4,34,1,21030.0,"""",table-completion,4,94816,0.0-4.0,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,ARWF605I7RWM7,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,YH7I1,YH7I1,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,334281,334281,GNG_0qmcw848,3I7DHKZYGO3UBYMLGJ0K9S9MC185F2,ARWF605I7RWM7,3MIVREZQVI1SYMZ4BKAIYW1TZSMKQ8,"""","""","""","""","""",[],"{""responses"":{""7"":{""name"":""7. Has thinking abo...",first-2,true,2021-12-04 04:32:47,173.23.42.244,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,WebKit,Chrome,96.0.4664.45,Linux,NaN,6356fe4a7118019579648d8725187f92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,11,1,6128.0,"""",table-completion,7,139651,0.0-7.0,3OKP4QVBP30V9KW4WXSHA4UVOTJGAU,A2ONILC0LZKG6Y,3K5TEWLKGWEVJJ3QNIOIGNGD5CRIVS,KE5K1,KE5K1,3K5TEWLKGWEVJJ3QNIOIGNGD5CRIVS,3K5TEWLKGWEVJJ3QNIOIGNGD5CRIVS,334281,334281,GNG_lqsppvhc,3K5TEWLKGWEVJJ3QNIOIGNGD5CRIVS,A2ONILC0LZKG6Y,3OKP4QVBP30V9KW4WXSHA4UVOTJGAU,"""","""","""","""","""",[],"{""responses"":{""20"":{""name"":""20. On what propor...",third_2,true,2021-12-03 15:06:16,47.135.1.239,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:9...,NaN,Firefox,94.0,Windows,10.0,6356fe4a7118019579648d8725187f92
8,11,1,29221.0,"""",table-completion,8,169379,0.0-8.0,3OKP4QVBP30V9KW4WXSHA4UVOTJGAU,A2ONILC0LZKG6Y,3K5TEWLKGWEVJJ3QNIOIGNGD5CRIVS,KE5K1,KE5K1

In [37]:

def clean_table_questions(x):
    dic = {}
    a = json.loads(x)['responses']
    for i in a:
        dic[i] = a[i]['choices'][0]
    return dic

def clean_form_questions(x):
    dic = {}
    for i in json.loads(x):
        dic[i['id']] = i['value']
    return dic

def clean_questions(x,type):
    if(type == 'table-completion'):
        return clean_table_questions(x)
    elif(type == 'custom-form'):
        return clean_form_questions(x)
    else:
        return x

In [38]:
questionnaire_respones['clean_responses'] =  questionnaire_respones.apply(lambda x: clean_questions(x['responses'],x['trial_type']), axis=1)

In [39]:
questionnaire_respones.clean_responses = questionnaire_respones.clean_responses.apply(lambda x: list(x.items()))

In [40]:
a = questionnaire_respones.groupby('mturkID').agg({'clean_responses': 'sum'}).reset_index()

In [41]:
a['clean_responses_dict'] = a.clean_responses.apply(lambda x: dict(x))

In [42]:
df_dict = {}

for i in a.iterrows():
    df_dict[(i[1]['mturkID'])] = i[1]['clean_responses_dict']

In [47]:
df = pd.DataFrame(df_dict).transpose().reset_index()

TypeError: cannot convert the series to <class 'int'>

0     11111
1     00000
2     00000
3     00600
4     20222
5     10100
6     00000
7     10000
8     00000
9     00000
10    00000
11    32233
12    10022
13    30233
14    00000
15    32433
16    33333
17    13011
18    00000
19    00000
20    10111
21    11100
22    00000
23    10000
24    11255
25    00000
26    10100
27    00000
28    21233
29    50444
30    20555
dtype: object

In [46]:
df.to_csv('./Data/cleaned_questionnaire_data.csv')